In [0]:
dbutils.widgets.text("p_file_date","2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
spark.sql(f"""
          create table if not exists f1_presentation.calculated_race_results(
          race_year int,
          team_name string,
          driver_id int,
          driver_name string,
          race_id int,
          position int,
          points int,
          calculated_points int,
          created_date TIMESTAMP,
          updated_date TIMESTAMP
          )
          using delta
    """)

DataFrame[]

In [0]:
spark.sql(f"""
                create or replace temp view race_result_updated
                as
                select races.race_year,
                constructors.name as team_name,
                drivers.driver_id,
                drivers.name as driver_name,
                races.race_id,
                results.position,
                results.points,
                11 - results.position as calculated_points
                from f1_processed.results
                join f1_processed.drivers on (drivers.driver_id = results.driver_id)
                join f1_processed.constructors on (constructors.constructor_id = results.constructor_id)
                join f1_processed.races on (races.race_id = results.race_id)
                where results.position <= 10 and results.file_date = '{v_file_date}'                 
          
    """)

DataFrame[]

In [0]:
%sql select * from race_result_updated ;

race_year,team_name,driver_id,driver_name,race_id,position,points,calculated_points
2021,Mercedes,1,Lewis Hamilton,1052,1,25.0,10
2021,Red Bull,815,Sergio Pérez,1052,5,10.0,6
2021,McLaren,817,Daniel Ricciardo,1052,7,6.0,4
2021,Mercedes,822,Valtteri Bottas,1052,3,16.0,8
2021,Red Bull,830,Max Verstappen,1052,2,18.0,9
2021,Ferrari,832,Carlos Sainz,1052,8,4.0,3
2021,Aston Martin,840,Lance Stroll,1052,10,1.0,1
2021,Ferrari,844,Charles Leclerc,1052,6,8.0,5
2021,McLaren,846,Lando Norris,1052,4,12.0,7
2021,AlphaTauri,852,Yuki Tsunoda,1052,9,2.0,2


In [0]:
spark.sql(f"""
        MERGE INTO f1_presentation.calculated_race_results tgt
        USING race_result_updated upd
        ON (tgt.driver_id = upd.driver_id and tgt.race_id = upd.race_id)
        WHEN MATCHED THEN
        UPDATE SET
            tgt.position = upd.position,
            tgt.points = upd.points,
            tgt.calculated_points = upd.calculated_points,
            tgt.updated_date = current_timestamp
        WHEN NOT MATCHED
        THEN INSERT (
            race_year,
            team_name,
            driver_id,
            driver_name,
            race_id,
            position,
            points,
            calculated_points,
            created_date
        )
        VALUES (
            race_year,
            team_name,
            driver_id,
            driver_name,
            race_id,
            position,
            points,
            calculated_points,
            current_timestamp
        )
   """)

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql select count(1) from race_result_updated;

count(1)
10


In [0]:
%sql select count(1) from f1_presentation.calculated_race_results;

count(1)
10055


In [0]:
# %sql use f1_processed;

In [0]:
# %sql
# create table f1_presentation.calculated_race_results
# using parquet
# as
# select races.race_year,
#        constructors.name as team_name,
#        drivers.name as driver_name,
#        results.position,
#        results.points,
#        11 - results.position as calculated_points
#   from f1_processed.results
#   join f1_processed.drivers on (drivers.driver_id = results.driver_id)
#   join f1_processed.constructors on (constructors.constructor_id = results.constructor_id)
#   join f1_processed.races on (races.race_id = results.race_id)
#   where results.position <= 10;

In [0]:
# %sql drop table f1_presentation.calculated_race_results;